## Final Project Applied Data Science Capstone

Author: Jose Javier Rueda Montes

### Part 1: Description of the problem and the data

#### Description of the problem

I am going to analyze the different neighborhoods of the city I live in, Madrid. I will cluster the neighborhoods according to the characteristics of their venues as restaurants, social places, parks or residential areas. Finally, I will be able to suggest where to open a business such as a restaurant or bar according to the type of food they serve.

This analysis will be interesting for business people who want to benefit from it to obtain a greater benefit in their investment.

#### Description of the data

The data will be obtained from several sources.

First of all, data from Madrid areas will be obtained by scraping the wikipedia webpage in which this information is shown. This websites is: https://es.wikipedia.org/wiki/Anexo:Barrios_administrativos_de_Madrid.

Then, the geospatial information will be uploaded manually from a website that provides the coordinates of any Google Maps site. This website is: https://geocode.localfocus.nl/.

Finally, with the Foursquare API, the neighborhoods will be explored so they can be clustered according to the venues sited in each of the districts.

### Part 2: Analysis of the data

#### Download of data

In [1]:
conda install -c anaconda beautifulsoup4

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    beautifulsoup4-4.9.0       |           py36_0         165 KB  anaconda
    ca-certificates-2020.1.1   |                0         132 KB  anaconda
    certifi-2020.4.5.1         |           py36_0         159 KB  anaconda
    openssl-1.1.1g             |       h7b6447c_0         3.8 MB  anaconda
    soupsieve-2.0.1            |             py_0          33 KB  anaconda
    ------------------------------------------------------------
                                           Total:         4.3 MB

The following NEW packages will be INSTALLED:

  beautifulsoup4     anaconda/linux-64::beautifulsoup4-4.9.0-py36_0
  soupsieve          anaconda/noarch::soupsieve-2.0.1-py_0

The following packag

In [2]:
import urllib.request
from bs4 import BeautifulSoup

In [3]:
neighs_url = 'https://es.wikipedia.org/wiki/Anexo:Barrios_administrativos_de_Madrid'
neighs_page = urllib.request.urlopen(neighs_url)
neighs_soup = BeautifulSoup(neighs_page)

In [4]:
neigh_table = neighs_soup.find('table', class_='wikitable sortable')
neigh_table

<table align="center" class="wikitable sortable">
<tbody><tr>
<th>Distrito
</th>
<th>Número
</th>
<th>Nombre
</th>
<th data-sort-type="number">Superficie (km²)<sup class="reference separada" id="cite_ref-2"><a href="#cite_note-2"><span class="corchete-llamada">[</span>2<span class="corchete-llamada">]</span></a></sup>​
</th>
<th>Imagen
</th></tr>
<tr>
<td align="center" rowspan="6"><a class="mw-redirect" href="/wiki/Distrito_Centro_(Madrid)" title="Distrito Centro (Madrid)">Centro</a><br/><a class="image" href="/wiki/Archivo:Centro_District_loc-map.svg"><img alt="Centro District loc-map.svg" data-file-height="527" data-file-width="527" decoding="async" height="200" src="//upload.wikimedia.org/wikipedia/commons/thumb/0/09/Centro_District_loc-map.svg/200px-Centro_District_loc-map.svg.png" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/0/09/Centro_District_loc-map.svg/300px-Centro_District_loc-map.svg.png 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/0/09/Centro_District_lo

In [5]:
import pandas as pd
import numpy as np

A=[]
B=[]
C=[]

for row in neigh_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==5:
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True))
    if len(cells)==4:
        A.append(A[-1])
        B.append(cells[0].find(text=True))
        C.append(cells[1].find(text=True))
        
df=pd.DataFrame(B,columns=['Code'])
df['District']=A
df['Neigh']=C
df["Code"]=df["Code"].str.replace('\n', '')
df.head(10)

,Code,District,Neigh
0,11,Centro,Palacio
1,12,Centro,Embajadores
2,13,Centro,Cortes
3,14,Centro,Justicia
4,15,Centro,Universidad
5,16,Centro,Sol
6,21,Arganzuela,Imperial
7,22,Arganzuela,Acacias
8,23,Arganzuela,Chopera
9,24,Arganzuela,Legazpi


In [6]:
df.shape

(131, 3)

In [7]:
geo_df = pd.read_csv('GeoinfoMadrid.csv', delimiter=';')
geo_df.head(10)

,Neighborhood,Latitude,Longitude
0,"Palacio, Madrid","40,416344","-3,710968"
1,"Embajadores, Madrid","40,407584","-3,700305"
2,"Cortes, Madrid","40,41284","-3,697598"
3,"Justicia, Madrid","40,423105","-3,693424"
4,"Universidad, Madrid","40,423612","-3,706059"
5,"Sol, Madrid, Spain","40,417717","-3,704072"
6,"Imperial, Madrid, Spain","40,406426","-3,716507"
7,"Acacias, Madrid, Spain","40,403122","-3,708383"
8,"Chopera, Madrid, Spain","40,39422","-3,698051"
9,"Legazpi, Madrid, Spain","40,391278","-3,690045"


In [8]:
geo_df.shape

(131, 3)

In [9]:
df_madrid = pd.concat([df, geo_df], axis=1, sort=False)
df_madrid.drop(columns=['Neighborhood'], inplace=True)
df_madrid.rename(columns={'Neigh':'Neighborhood'}, inplace=True)
df_madrid = df_madrid.apply(lambda x: x.str.replace(',','.'))
df_madrid['Latitude'] = df_madrid['Latitude'].astype(float)
df_madrid['Longitude'] = df_madrid['Longitude'].astype(float)
df_madrid['Code'] = df_madrid['Code'].astype(int)
df_madrid.head()

,Code,District,Neighborhood,Latitude,Longitude
0,11,Centro,Palacio,40.416344,-3.710968
1,12,Centro,Embajadores,40.407584,-3.700305
2,13,Centro,Cortes,40.412840,-3.697598
3,14,Centro,Justicia,40.423105,-3.693424
4,15,Centro,Universidad,40.423612,-3.706059


In [10]:
print("Madrid has {} districts and {} neighborhoods.".format(len(df_madrid["District"].unique()), df_madrid.shape[0] ))

Madrid has 21 districts and 131 neighborhoods.


In [11]:
latitude = 40.416775
longitude = -3.703790

print("The geographical coordinates of Madrid city are: ({}, {}).".format(latitude, longitude))

The geographical coordinates of Madrid city are: (40.416775, -3.70379).


In [12]:
df_madrid.dtypes

Code              int64
District         object
Neighborhood     object
Latitude        float64
Longitude       float64
dtype: object

#### Plotting the data

In [13]:
import folium
map_madrid = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, district, neighborhood in zip(df_madrid['Latitude'], df_madrid['Longitude'], df_madrid['District'], df_madrid['Neighborhood']):
    label = '{}, {}'.format(neighborhood, district)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_madrid)  
    
map_madrid

#### Getting the venues with Foursquare

In [14]:
CLIENT_ID = 'YKSDVE0L1BZMZX3IXXVR2FHHO0D0R31XOERW331ZSG1B3ZAI' # your Foursquare ID
CLIENT_SECRET = '1YBIHQMGBFZNLP1RZJZSR0N21WXXRPBSAHIZNCYH2RYSBTXT' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100
radius = 700

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: YKSDVE0L1BZMZX3IXXVR2FHHO0D0R31XOERW331ZSG1B3ZAI
CLIENT_SECRET:1YBIHQMGBFZNLP1RZJZSR0N21WXXRPBSAHIZNCYH2RYSBTXT


In [15]:
import json
from pandas.io.json import json_normalize
import requests

In [16]:
def getNearbyVenues(names, latitudes, longitudes, radius):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [17]:
madrid_venues = getNearbyVenues(names=df_madrid['Neighborhood'],
                                   latitudes=df_madrid['Latitude'],
                                   longitudes=df_madrid['Longitude'],
                                   radius=radius
                                  )

Palacio
Embajadores
Cortes
Justicia
Universidad
Sol
Imperial
Acacias
Chopera
Legazpi
Delicias
Palos de Moguer
Atocha
Pacífico
Adelfas
Estrella
Ibiza
Jerónimos
Niño Jesús
Recoletos
Goya
Fuente del Berro
La Guindalera
Lista
Castellana
El Viso
Prosperidad
Ciudad Jardín
Hispanoamérica
Nueva España
Castilla
Bellas Vistas
Cuatro Caminos
Castillejos
Almenara
Valdeacederas
Berruguete
Gaztambide
Arapiles
Trafalgar
Almagro
Ríos Rosas
Vallehermoso
El Pardo
Fuentelarreina
Peñagrande
El Pilar
La Paz
Valverde
Mirasierra
El Goloso
Casa de Campo
Argüelles
Ciudad Universitaria
Valdezarza
Valdemarín
El Plantío
Aravaca
Los Cármenes
Puerta del Ángel
Lucero
Aluche
Campamento
Cuatro Vientos
Las Águilas
Comillas
Opañel
San Isidro
Vista Alegre
Puerta Bonita
Buenavista
Abrantes
Orcasitas
Orcasur
San Fermín
Almendrales
Moscardó
Zofío
Pradolongo
Entrevías
San Diego
Palomeras Bajas
Palomeras Sureste
Portazgo
Numancia
Pavones
Horcajo
Marroquina
Media Legua
Fontarrón
Vinateros
Ventas
Pueblo Nuevo
Quintana
Concepció

In [18]:
madrid_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Palacio,40.416344,-3.710968,Cervecería La Mayor,40.415218,-3.712194,Beer Bar
1,Palacio,40.416344,-3.710968,Teatro Real de Madrid,40.418226,-3.711064,Opera House
2,Palacio,40.416344,-3.710968,la gastroteca de santiago,40.416639,-3.710944,Restaurant
3,Palacio,40.416344,-3.710968,Plaza de Oriente,40.418326,-3.712196,Plaza
4,Palacio,40.416344,-3.710968,Zuccaru,40.417179,-3.711674,Ice Cream Shop


In [19]:
madrid_venues.shape

(5535, 7)

In [20]:
madrid_venues.groupby("Neighborhood").count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Abrantes,10,10,10,10,10,10
Acacias,75,75,75,75,75,75
Adelfas,75,75,75,75,75,75
Aeropuerto,65,65,65,65,65,65
Alameda de Osuna,26,26,26,26,26,26
...,...,...,...,...,...,...
Ventas,17,17,17,17,17,17
Villaverde Alto,6,6,6,6,6,6
Vinateros,19,19,19,19,19,19


In [21]:
print('There are {} uniques categories.'.format(len(madrid_venues['Venue Category'].unique())))

There are 296 uniques categories.


In [22]:
min_venue = 10
(madrid_venues.groupby("Neighborhood").count()["Venue"] > min_venue).value_counts()
print('There are {} neighborhoods which have less than {} venues.'.format((madrid_venues.groupby("Neighborhood").count()["Venue"] > min_venue).value_counts()[0], min_venue))

There are 18 neighborhoods which have less than 10 venues.


In [23]:
print("The radius for each neighborhood has been {} meters and the maximum number of venues retrieved per neighborhood has been {}.".format(radius, LIMIT))

The radius for each neighborhood has been 700 meters and the maximum number of venues retrieved per neighborhood has been 100.


#### Preparing the data for clustering

In [24]:
# one hot encoding
madrid_onehot = pd.get_dummies(madrid_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
madrid_onehot['Neighborhoods'] = madrid_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [madrid_onehot.columns[-1]] + list(madrid_onehot.columns[:-1])
madrid_onehot = madrid_onehot[fixed_columns]

madrid_onehot.head()

,Neighborhoods,Accessories Store,Adult Boutique,Airport,Airport Lounge,American Restaurant,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,...,Used Bookstore,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Palacio,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Palacio,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Palacio,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Palacio,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Palacio,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
madrid_grouped = madrid_onehot.groupby('Neighborhoods').mean().reset_index()
madrid_grouped.head()

,Neighborhoods,Accessories Store,Adult Boutique,Airport,Airport Lounge,American Restaurant,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,...,Used Bookstore,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Abrantes,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
1,Acacias,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.053333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.013333,0.0,0.0
2,Adelfas,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
3,Aeropuerto,0.0,0.0,0.0,0.0,0.015385,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
4,Alameda de Osuna,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0


#### Obtaining the most common venues per neighborhood

In [26]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [27]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhoods']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhoods'] = madrid_grouped['Neighborhoods']

for ind in np.arange(madrid_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(madrid_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Abrantes,Gym / Fitness Center,Fast Food Restaurant,Bakery,Park,Athletics & Sports
1,Acacias,Spanish Restaurant,Bar,Tapas Restaurant,Restaurant,Art Gallery
2,Adelfas,Bar,Bakery,Spanish Restaurant,Gym,Grocery Store
3,Aeropuerto,Clothing Store,Hotel,Italian Restaurant,Restaurant,Tapas Restaurant
4,Alameda de Osuna,Park,Hotel,Spanish Restaurant,Restaurant,Italian Restaurant


#### Clustering neighborhoods

In [28]:
from sklearn.cluster import KMeans

In [29]:
# set number of clusters
kclusters = 8

madrid_grouped_clustering = madrid_grouped.drop('Neighborhoods', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0)
kmeans.fit(madrid_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([7, 1, 1, 1, 1, 0, 0, 4, 1, 0], dtype=int32)

In [63]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

madrid_merged = df_madrid

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
madrid_merged = madrid_merged.join(neighborhoods_venues_sorted.set_index('Neighborhoods'), on='Neighborhood')

madrid_merged.head() # check the last columns!

,Code,District,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,11,Centro,Palacio,40.416344,-3.710968,0,Tapas Restaurant,Plaza,Spanish Restaurant,Hotel,Restaurant
1,12,Centro,Embajadores,40.407584,-3.700305,0,Café,Restaurant,Plaza,Spanish Restaurant,Tapas Restaurant
2,13,Centro,Cortes,40.412840,-3.697598,0,Hotel,Plaza,Art Museum,Hostel,Theater
3,14,Centro,Justicia,40.423105,-3.693424,0,Restaurant,Spanish Restaurant,Hotel,Plaza,Café
4,15,Centro,Universidad,40.423612,-3.706059,0,Hotel,Plaza,Café,Bookstore,Tapas Restaurant


#### Plotting the clusters

In [31]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

In [32]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(madrid_merged['Latitude'], madrid_merged['Longitude'], madrid_merged['Neighborhood'], madrid_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Tagging the clusters

In [34]:
hotels = madrid_merged.loc[madrid_merged['Cluster Labels'] == 0, madrid_merged.columns[[2] + list(range(5, madrid_merged.shape[1]))]]

In [36]:
food = madrid_merged.loc[madrid_merged['Cluster Labels'] == 1, madrid_merged.columns[[2] + list(range(5, madrid_merged.shape[1]))]]

In [38]:
mirasierra = madrid_merged.loc[madrid_merged['Cluster Labels'] == 2, madrid_merged.columns[[2] + list(range(5, madrid_merged.shape[1]))]]

In [40]:
cañaveral = madrid_merged.loc[madrid_merged['Cluster Labels'] == 3, madrid_merged.columns[[2] + list(range(5, madrid_merged.shape[1]))]]

In [42]:
spanish_rests = madrid_merged.loc[madrid_merged['Cluster Labels'] == 4, madrid_merged.columns[[2] + list(range(5, madrid_merged.shape[1]))]]

In [44]:
sports = madrid_merged.loc[madrid_merged['Cluster Labels'] == 5, madrid_merged.columns[[2] + list(range(5, madrid_merged.shape[1]))]]

In [46]:
casacampo = madrid_merged.loc[madrid_merged['Cluster Labels'] == 6, madrid_merged.columns[[2] + list(range(5, madrid_merged.shape[1]))]]

In [48]:
residential = madrid_merged.loc[madrid_merged['Cluster Labels'] == 7, madrid_merged.columns[[2] + list(range(5, madrid_merged.shape[1]))]]

In [49]:
print('Madrid neighborhoods can be clustered into {} areas according to the {} most common venues in each of them.'.format(kclusters, num_top_venues))

Madrid neighborhoods can be clustered into 8 areas according to the 5 most common venues in each of them.


In [61]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []

madrid_merged = madrid_merged[madrid_merged['Cluster Labels']==4]

for lat, lon, poi, cluster in zip(madrid_merged['Latitude'], madrid_merged['Longitude'], madrid_merged['Neighborhood'], madrid_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters